In [1]:
# Import necessary packages
import ee
import geemap

Helpful resource: https://gis.stackexchange.com/questions/326131/error-in-export-image-from-google-earth-engine-from-python-api

In [2]:
# Initiate interactive map
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap('HYBRID') # Add Google Map
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# If you want to define a different ROI with inspector tool
# roi = ee.FeatureCollection(Map.draw_features)

# Define ROI (this should be updated)
roi = ee.Geometry.Polygon([[[-104.617064, 39.597157],
                            [-104.617064, 39.747256],
                            [-104.573116, 39.835893],
                            [-104.504448, 40.000202],
                            [-104.644286, 40.056921],
                            [-104.858778, 40.008618],
                            [-104.993368, 39.785257],
                            [-104.957436, 39.66901],
                            [-104.845047, 39.618318],
                            [-104.836804, 39.61197],
                            [-104.617064, 39.597157]]])

Map.addLayer(roi)

#### Get Landsat8 Spectral Unmix ee.Image object with an roi 

In [28]:
# Function to get cloud free summer composite for given year
def ppUnmix(year, ROI):
    """pp Unmix will Pre Process landsat and apply spectral unmixing to produce fractions of % urban."""
    # Get raw Landsat8 data input for simple composite
    if year >= 2013:
        landsatCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1')

    elif 1995 <= year < 2013:
        landsatCollection = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')
        print('Function does not work with Landsat 5 yet (before 2013)')

    else:
        print('Year is out of range: 1995 - 2015.  This function will fail.')

    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-07-01', str(year)+'-09-30'

    # Filter Landsat8 collection by date and by ROI
    collection = landsatCollection.filterDate(d1, d2).filterBounds(ROI)

    # Create cloud-free composite with default params
    composite = ee.Algorithms.Landsat.simpleComposite(collection)

    image = composite.clip(ROI)

    # Select bands 1-7 and clip to ROI
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    image = image.select(bands)

    # End-members from GEE unmix() example script here:
    # https://developers.google.com/earth-engine/image_transforms
    urban = [88, 42, 48, 38, 86, 115, 59]  # band_0
    veg = [50, 21, 20, 35, 50, 110, 23]  # band_1
    water = [51, 20, 14, 9, 7, 116, 4]  # band_2

    # Calculate per-pixel fractions at each pixel of each of the end-members
    fractions = image.unmix([urban, veg, water])

    return fractions

In [23]:
xx = cloudFreeComposite(1995, roi)
yy = cloudFreeComposite(2013, roi)

Function does not work with Landsat 5 yet (before 2013)


In [30]:
dict = {}
for i in range(2013, 2016):
    dict[str(i)] = ppUnmix(year=i, ROI=roi)
    

In [16]:
Map.addLayer(yy, {}, 'fractions')
Map

Map(center=[39.87865414303291, -104.6825408935547], controls=(WidgetControl(options=['position'], widget=HBox(…

In [17]:
yy.getInfo()

{'type': 'Image',
 'bands': [{'id': 'band_0',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 2],
   'origin': [-105, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'band_1',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 2],
   'origin': [-105, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'band_2',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 2],
   'origin': [-105, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

### Example from docs: unmix
* this is trying to solve the issue that arises when I try and visualize the spectrally unmixed Landsat 5 ee.Image object.
* I should try and visualize the RGB first.
* error code: 

EEException: ImageCollection.reduce: Error in map(ID=LT05_033032_20100716):
Landsat.TOA: Band 'B1' is Type<Float>, expected Type<Short<0, 255>>.

In [21]:
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA').filterDate('2010-07-01','2010-09-30').filterBounds(roi)

composite = ee.Algorithms.Landsat.simpleComposite(l5)

image = composite.select(bands)#.clip(roi)

# image = ee.Image('LANDSAT/LT05/C01/T1/LT05_044034_20080214').select(bands)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128}, 'image')

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])
Map.addLayer(fractions, {}, 'unmixed')

EEException: ImageCollection.reduce: Error in map(ID=LT05_033032_20100716):
Landsat.TOA: Band 'B1' is Type<Float>, expected Type<Short<0, 255>>.

In [ ]:
# Get  L8 image collection 2013
l813 = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2013-07-01','2013-09-30').filterBounds(roi)

### Next up:
* learn how to clip to ROI
    * important to make smaller before exporting
* export fractions tiff (and rgb??)
    * once exported fix the fractions to not include negatives
        * this is because built-in unmix() method does not use the FCLS calculation
* understand how to export for each year (Landsat 5 and Landsat 8, 1995 to 2015)


### Function to get a Landsat8 ee.Image object for the given ROI, cloud free and clipped

In [ ]:
# Get collection of Landsat scenes for the dates and region you wish
def getLandsat8(year, ROI):
    # Get raw Landsat8 data input for simple composite
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
    
    # Generate date strings for .filterDate()
    d1, d2 = str(year)+'-07-01', str(year)+'-09-30'

    # Filter Landsat8 collection by date and by ROI
    collection = l8.filterDate(d1, d2).filterBounds(ROI)
 
    # Create cloud-free composite with default params
    composite = ee.Algorithms.Landsat.simpleComposite(collection)
    
    # Clip to area (provided ROI)
    # Returns a single image, not a collection
    # because this function is flawed
    return composite.clip(ROI)


In [ ]:
for i in range(2013, 2014):
    image = getLandsat8(year=i, ROI=roi)

## Export with built-in Export.todrive functions
* Successful!  
* Needs some parameter tuning.  Specifically the folder name that is generated and the file name.
* **See for solutions:** https://gis.stackexchange.com/questions/326131/error-in-export-image-from-google-earth-engine-from-python-api

In [39]:
task_config = {
    'fileNamePrefix': 'fractions_2013',
    'crs': 'EPSG:4326',
    'scale': 500,
    'maxPixels': 1000000000000,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': roi,
    'folder': 'myfolder'
}

task = ee.batch.Export.image.toDrive(dict['2013'], str('fractions_2013'), **task_config)
task.start()

In [40]:
task_config = {
    'fileNamePrefix': 'fractions_2014',
    'crs': 'EPSG:4326',
    'scale': 500,
    'maxPixels': 1000000000000,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': roi,
    'folder': 'myfolder'
}

task = ee.batch.Export.image.toDrive(dict['2014'], str('fractions_2014'), **task_config)
task.start()

In [41]:
task_config = {
    'fileNamePrefix': 'fractions_2015',
    'crs': 'EPSG:4326',
    'scale': 500,
    'maxPixels': 1000000000000,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': roi,
    'folder': 'myfolder'
}

task = ee.batch.Export.image.toDrive(dict['2015'], str('fractions_2015'), **task_config)
task.start()

In [ ]:
def exportL8_toGglDrive(IMAGE, DESCRIPTION):
    ee.Export.image.toDrive(image=IMAGE,
                             description=DESCRIPTION, 
                             folder='GEE_Landsat8_Exports')
    

In [ ]:
exportL8_toGglDrive(image, 'GEE_export_test1')

Export.image.toDrive(image, description,
                     folder, fileNamePrefix,
                     dimensions, region, scale,
                     crs,
                     crsTransform, maxPixels,
                     shardSize, fileDimensions,
                     skipEmptyTiles, fileFormat, formatOptions)